In [55]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd


options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(executable_path='C:/Code/chromedriver/chromedriver.exe')

In [91]:
def get_reviews(driver):
    comments = driver.find_elements_by_class_name('raw__373c0__3rcx7')

    comments_text = []
    for comment in comments:
        comments_text.append(comment.text)

    return comments_text[5:]

In [92]:
def extract_comments(driver, link, num_pages):
    driver.get(link);
    time.sleep(5) 
    
    csv = pd.DataFrame(get_reviews(driver))

    for i in range(num_pages-1):
        button = driver.find_element_by_xpath('/html/body/div[@*]/div[@*]/yelp-react-root/div/div[@*]/div/div/div[@*]/div/div[@*]/div[@*]/section[@*]/div[@*]/div/div[@*]/div[@*]/div/div[last()]/span/a')
        button.click()

        time.sleep(5)

        csv_new_page = pd.DataFrame(get_reviews(driver))
        csv = pd.concat([csv, csv_new_page], ignore_index=True)
        
    return csv

In [87]:
links = {
    'air-france': 'https://www.yelp.com/biz/air-france-san-francisco', 
     #'hop': '',
    'klm-royal-dutch-airlines': 'https://www.yelp.com/biz/klm-royal-dutch-airlines-amstelveen',
     #'transavia': '',
    'lufthansa': 'https://www.yelp.com/biz/lufthansa-frankfurt-am-main-3',
    #'swiss-international-air-lines': '',
    'ryanair': 'https://www.yelp.com/biz/ryanair-dublin',
    'easyjet': 'https://www.yelp.com/biz/easyjet-luton',
    'british-airways': 'https://www.yelp.com/biz/british-airways-west-drayton',
    #'iberia': '',
    'vueling-airlines': 'https://www.yelp.com/biz/vueling-barcelona',
    #'level': '',
    'singapore-airlines': 'https://www.yelp.com/biz/singapore-airlines-san-francisco-7',
    #'qatar-airways': '',
    #'emirates': '',
    'american-airlines': 'https://www.yelp.com/biz/american-airlines-los-angeles-5',
    'united': 'https://www.yelp.com/biz/united-airlines-los-angeles-5',
    'southwest' : 'https://www.yelp.com/biz/southwest-airlines-chicago-2'
}

In [104]:
# It is ideal to manually input the desired airline and number of pages
# According to the quantity of available reviews + the desire of maintenance
airline = 'southwest'
df = extract_comments(driver, links[airline], 10)

df['airline'] = airline
df.columns = ['review', 'airline']
df.to_csv('yelp_' + airline + '.csv', sep='|')


In [105]:
driver.quit()

After generation of the desired extractions, the data can be merged into a single dataframe if needed. 

In [113]:
import glob
path = r"."                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
print(all_files)
li = []
for filename in all_files:
    print(filename)
    df = pd.read_csv(filename, sep="|")
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

['.\\yelp_air-france.csv', '.\\yelp_american-airlines.csv', '.\\yelp_british-airways.csv', '.\\yelp_easyjet.csv', '.\\yelp_klm-royal-dutch-airlines.csv', '.\\yelp_lufthansa.csv', '.\\yelp_ryanair.csv', '.\\yelp_singapore-airlines.csv', '.\\yelp_southwest.csv', '.\\yelp_united.csv', '.\\yelp_vueling-airlines.csv']
.\yelp_air-france.csv
.\yelp_american-airlines.csv
.\yelp_british-airways.csv
.\yelp_easyjet.csv
.\yelp_klm-royal-dutch-airlines.csv
.\yelp_lufthansa.csv
.\yelp_ryanair.csv
.\yelp_singapore-airlines.csv
.\yelp_southwest.csv
.\yelp_united.csv
.\yelp_vueling-airlines.csv


In [118]:
frame.columns = ['original_id', 'review', 'airline']
output = frame.drop(['original_id'], axis=1)
output.to_csv("yelp_merged.csv", sep='|')